In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Faster Sentiment Analysis
This notebook follows this tutorial: https://github.com/bentrevett/pytorch-sentiment-analysis.

In [2]:
def append_bigrams(x):
    bigrams = set(["%s %s" % (f, s) for f, s in zip(x[:-1], x[1:])])
    x.extend(list(bigrams))
    return x

append_bigrams("This film is the best".split())

['This',
 'film',
 'is',
 'the',
 'best',
 'is the',
 'film is',
 'the best',
 'This film']

In [3]:
import torchtext
from torchtext.data import Field, LabelField, BucketIterator

TEXT = Field(tokenize='spacy', preprocessing=append_bigrams, include_lengths=True)
LABEL = LabelField(dtype=torch.float)

train_data, test_data = torchtext.datasets.IMDB.splits(TEXT, LABEL, root='data')
train_data, valid_data = train_data.split()

In [4]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE, 
                 vectors="glove.6B.100d", vectors_cache="vector_cache", 
                 unk_init=torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [5]:
# The bi-grams are included in the `Dataset`.
print(train_data[0].text)
# The bi-grams are also included in the `TEXT.vocab`.
print(TEXT.vocab.itos[-10:])

['One', 'does', "n't", 'get', 'to', 'enjoy', 'this', 'gem', ',', 'the', '1936', 'Invisible', 'Ray', ',', 'often', '.', 'But', 'no', 'can', 'forget', 'it', '.', 'The', 'story', 'is', 'elegant', '.', 'Karloff', ',', 'austere', 'and', 'embittered', 'in', 'his', 'Carpathian', 'mountain', 'retreat', ',', 'is', 'Janos', 'Rukh', ',', 'genius', 'science', 'who', 'reads', 'ancient', 'beams', 'of', 'light', 'to', 'ascertain', 'events', 'in', 'the', 'great', 'geological', 'past', '\x85', 'particularly', 'the', 'crash', 'of', 'a', 'potent', 'radioactive', 'meteor', 'in', 'Africa', '.', 'Joining', 'him', 'is', 'the', 'ever', '-', 'elegant', 'Lugosi', '(', 'as', 'a', 'rare', 'hero', ')', ',', 'who', 'studies', '"', 'astro', '-', 'chemistry', '.', '"', 'Frances', 'Drake', 'is', 'the', 'lovely', ',', 'underused', 'young', 'wife', ';', 'Frank', 'Lawton', 'the', 'romantic', 'temptation', ';', 'and', 'the', 'divine', 'Violet', 'Kemble', 'Cooper', 'is', 'Mother', 'Rukh', ',', 'in', 'a', 'performance', 'wo

In [6]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, device=device)

In [7]:
for batch in train_iterator:
    break
text, text_lens = batch.text
print(text)
print(text_lens)

tensor([[ 4202,   351,  2367,  ...,    11,    11,  3447],
        [ 2559,     6,  4158,  ...,   815,   229, 21850],
        [    5,  1966, 23814,  ...,    11,    16,  1379],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]], device='cuda:0')
tensor([ 235,  498,  957,  709,  281,  326,  294,  404,  616,  475,  656,  598,
         335,  332,  336,  669, 2071,  961,  312,  525,  537,  273,  242,  517,
         573,  290,  572,  501,  303,  739,   95,   61,  582,  813,  628,  249,
         459,  400,  254,  205,  325,  290,  201, 1083,  181,  696,  270,  492,
         365,  780,  970, 1056,  339,  373,  235,  840,  369,   92,  398,  482,
         575,  778,  286,  687], device='cuda:0')


# Build Model

In [8]:
class Classifier(nn.Module):
    def __init__(self, in_dim, emb_dim, out_dim, pad_idx):
        super().__init__()
        self.emb = nn.Embedding(in_dim, emb_dim, padding_idx=pad_idx)
        self.fc = nn.Linear(emb_dim, out_dim)

    def forward(self, text, text_lens):
        # text: (step, batch)
        embedded = self.emb(text)
        
        # Pooling along steps
        # pooled: (batch, emb)
        pooled = embedded.sum(dim=0) / text_lens.unsqueeze(1)
        return self.fc(pooled)

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

IN_DIM = len(TEXT.vocab)
EMB_DIM = 100
OUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

classifier = Classifier(IN_DIM, EMB_DIM, OUT_DIM, PAD_IDX).to(device)
count_parameters(classifier)

2500301

In [10]:
# Initialize Embeddings with Pre-Trained Vectors
classifier.emb.weight.data.copy_(TEXT.vocab.vectors)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

classifier.emb.weight.data[UNK_IDX].zero_()
classifier.emb.weight.data[PAD_IDX].zero_()

print(classifier.emb.weight[:5, :8])

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344],
        [-0.1077,  0.1105,  0.5981, -0.5436,  0.6740,  0.1066,  0.0389,  0.3548],
        [-0.3398,  0.2094,  0.4635, -0.6479, -0.3838,  0.0380,  0.1713,  0.1598]],
       device='cuda:0', grad_fn=<SliceBackward>)


# Train Model

In [11]:
optimizer = optim.Adam(classifier.parameters())
# Binary cross entropy with logits. 
# The binary version of cross entropy loss. 
loss_func = nn.BCEWithLogitsLoss().to(device)

In [12]:
def train_epoch(classifier, iterator, optimizer, loss_func):
    classifier.train()
    epoch_loss = 0
    epoch_acc = 0
    for batch in iterator:
        # Forward pass
        text, text_lens = batch.text
        preds = classifier(text, text_lens).squeeze(-1)
        # Calculate loss
        loss = loss_func(preds, batch.label)
        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        # Update weights
        optimizer.step()
        # Accumulate loss and acc
        epoch_loss += loss.item()
        epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

def eval_epoch(classifier, iterator, loss_func):
    classifier.eval()
    epoch_loss = 0
    epoch_acc = 0
    with torch.no_grad():
        for batch in iterator:
            # Forward pass
            text, text_lens = batch.text
            preds = classifier(text, text_lens).squeeze(-1)
            # Calculate loss
            loss = loss_func(preds, batch.label)
            # Accumulate loss and acc
            epoch_loss += loss.item()
            epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [13]:
import time
N_EPOCHS = 5
best_valid_loss = np.inf

for epoch in range(N_EPOCHS):
    t0 = time.time()
    train_loss, train_acc = train_epoch(classifier, train_iterator, optimizer, loss_func)
    valid_loss, valid_acc = eval_epoch(classifier, valid_iterator, loss_func)
    epoch_secs = time.time() - t0

    epoch_mins, epoch_secs = int(epoch_secs // 60), int(epoch_secs % 60)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(classifier.state_dict(), 'models/tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 8s
	Train Loss: 0.672 | Train Acc: 64.65%
	 Val. Loss: 0.635 |  Val. Acc: 73.87%
Epoch: 02 | Epoch Time: 0m 11s
	Train Loss: 0.556 | Train Acc: 79.72%
	 Val. Loss: 0.496 |  Val. Acc: 82.27%
Epoch: 03 | Epoch Time: 0m 13s
	Train Loss: 0.415 | Train Acc: 85.98%
	 Val. Loss: 0.398 |  Val. Acc: 85.66%
Epoch: 04 | Epoch Time: 0m 11s
	Train Loss: 0.327 | Train Acc: 89.04%
	 Val. Loss: 0.346 |  Val. Acc: 86.87%
Epoch: 05 | Epoch Time: 0m 11s
	Train Loss: 0.273 | Train Acc: 90.82%
	 Val. Loss: 0.317 |  Val. Acc: 87.76%


In [14]:
classifier.load_state_dict(torch.load('models/tut3-model.pt'))

valid_loss, valid_acc = eval_epoch(classifier, valid_iterator, loss_func)
test_loss, test_acc = eval_epoch(classifier, test_iterator, loss_func)

print(f'Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}%')
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Val. Loss: 0.317 | Val. Acc: 87.76%
Test Loss: 0.315 | Test Acc: 87.75%


# User Input

In [15]:
import spacy
nlp = spacy.load('en')

def predict_senti(classifier, sentence):
    classifier.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[tok] for tok in tokenized]
    lens = len(indexed)

    indexed = torch.tensor(indexed, dtype=torch.long).unsqueeze(1).to(device)
    lens = torch.tensor(lens, dtype=torch.long).unsqueeze(0).to(device)
    pred = torch.sigmoid(classifier(indexed, lens)).round().type(torch.long)
    return LABEL.vocab.itos[pred.item()]

In [16]:
predict_senti(classifier, "This is a good film.")

'pos'

In [17]:
predict_senti(classifier, "This film is terrible.")

'neg'

# Check Embeddings
* The Embeddings of `unk` and `<pad>` tokens
    * Because the `padding_idx` has been passed to `nn.Embedding`, so the `<pad>` embedding will remain zeros throughout training.  
    * While the `<unk>` embedding will be learned.

In [18]:
print(classifier.emb.weight[:5, :8])

tensor([[-0.0263,  0.0257, -0.0321, -0.0201, -0.0059,  0.0239,  0.0258,  0.0236],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0313, -0.3120,  0.7726, -0.3239,  0.1757, -0.0388, -0.4670,  0.2690],
        [-0.0344,  0.0401,  0.6453, -0.4638,  0.7781,  0.0203, -0.0433,  0.2862],
        [-0.3167,  0.1872,  0.4716, -0.6247, -0.3226,  0.0084,  0.1469,  0.1400]],
       device='cuda:0', grad_fn=<SliceBackward>)
